# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from smart_open import open
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [5]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [6]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [7]:
model_nums = config["clustering"]["LDA"]["max_model_num"]

In [8]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)
        self.dictionary.filter_extremes()

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)
                
    def __len__(self):
        return len(self.texts)

In [9]:
texts = newsgroups_df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)
dictionary.filter_extremes()

100%|██████████| 18846/18846 [00:00<00:00, 54762.66it/s]


In [10]:
os.makedirs(os.path.dirname("../../data/20News/LDA/"), exist_ok=True)
pickle.dump(dictionary, open("../../data/20News/LDA/dictionary.sav", "wb"))
pickle.dump(corpus, open("../../data/20News/LDA/corpus.sav", "wb"))

In [11]:
label = newsgroups_df["class"].to_numpy()

In [12]:
def getLDA(corpus,dictionary, n_components, seed, path):
    lda = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=n_components,
        alpha="auto",
        eval_every=5,
        random_state=seed,
    )
    # save model
    os.makedirs(os.path.dirname(path), exist_ok=True)
    lda.save(path)
    pred = [lda[docBow] for docBow in corpus]
    return pred, lda

In [13]:
n_conmponents=20

In [14]:
models_path = "../../data/20News/LDA/model/"
pred_path = "../../data/20News/LDA/pred/"

for model_num in tqdm(range(model_nums)):
    prob, lda = getLDA(
        corpus=corpus,
        dictionary=dictionary,
        n_components=n_conmponents,
        seed=model_num,
        path=f"{models_path}{model_num}"
    )
#     save prediction
    probDf = pd.DataFrame([dict(row) for row in prob]).fillna(0)
    pred = probDf.idxmax(axis=1).to_numpy()
    os.makedirs(f"{pred_path}", exist_ok=True)
    with open(
        f"{pred_path}{model_num}.csv", "w"
    ) as f:
        writer = csv.writer(f)
        writer.writerow(pred)

100%|██████████| 2/2 [01:04<00:00, 32.42s/it]


In [15]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(
      lda, corpus, dictionary, n_jobs = 1, sort_topics = False
      )

pyLDAvis.display(vis)

/opt/conda/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [16]:
newsgroups_df["pred"]  = pred

In [17]:
newsgroups_df[newsgroups_df["pred"]==1].text.to_csv("temp2.csv")

In [18]:
newsgroups_df[newsgroups_df["words_nonstop"].str.contains("'AX")]

,text,filenames,class,words,words_nonstop,pred
1800,Subject: roman 03/14\nFrom: pwiseman@salmon.us...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman 03/14 From : pwiseman @ salmon...,Subject roman 03/14 pwiseman salmon.usd.edu Cl...,10
2694,Subject: roman.bmp 09/14------------ Part 9 of...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 09/14 -- -- -- -- -- -- Pa...,Subject roman.bmp 09/14 Part 14 pwiseman salmo...,10
3059,Subject: roman.bmp 04/14\nFrom: pwiseman@salmo...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 04/14 From : pwiseman @ sa...,Subject roman.bmp 04/14 pwiseman salmon.usd.ed...,10
5017,Subject: roman.bmp 11/14 \nFrom: pwiseman@salm...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 11/14 From : pwiseman @ sa...,Subject roman.bmp 11/14 pwiseman salmon.usd.ed...,10
5396,Subject: roman.bmp 14/14\nFrom: pwiseman@salmo...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 14/14 From : pwiseman @ sa...,Subject roman.bmp 14/14 pwiseman salmon.usd.ed...,10
5925,From: mitchell@nodecg.ncc.telecomwa.oz.au (Cli...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,From : mitchell @ nodecg.ncc.telecomwa.oz.au (...,mitchell nodecg.ncc.telecomwa.oz.au Clive Mitc...,10
6872,Subject: roman 02/14 \nFrom: pwiseman@salmon.u...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman 02/14 From : pwiseman @ salmon...,Subject roman 02/14 pwiseman salmon.usd.edu Cl...,10
7829,Subject: roman.bmp 13/14\nFrom: pwiseman@salmo...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 13/14 From : pwiseman @ sa...,Subject roman.bmp 13/14 pwiseman salmon.usd.ed...,10
8452,Subject: roman.bmp 07/14\nFrom: pwiseman@salmo...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.os.ms-windows.misc,Subject : roman.bmp 07/14 From : pwiseman @ sa...,Subject roman.bmp 07/14 pwiseman salmon.usd.ed...,10
8709,From: dcr@mail.ast.cam.ac.uk (Derek C. Richard...,/home/jovyan/scikit_learn_data/20news_home/20n...,comp.windows.x,From : dcr @ mail.ast.cam.ac.uk ( Derek C. Ric...,dcr mail.ast.cam.ac.uk Derek C. Richardson Sub...,10
